In [1]:
!pip install -q rouge_score
!pip install -q evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.9 MB/s eta 0:00:00


In [2]:
import statistics
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
import evaluate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
# finetuned data
df_test = pd.read_csv("your_data")

In [37]:
df_test = df_test.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df_test.head()

,MWE,Previous,Target,Next,Label,Context,All,Generated Next
0,sangue quente,E é claro aproveitou a parada para conversar e...,"Após o jogo, o português de uma pequena amostr...",Entrou em campo para reclamar e cobrar o árbit...,0,E é claro aproveitou a parada para conversar e...,E é claro aproveitou a parada para conversar e...,"Ele deu algumas dicas para o jovem atacante, i..."
1,buraco negro,O achado foi publicado nesta quarta-feira (21)...,Os autores da pesquisa apelidaram o buraco neg...,Anel de poeira na órbita de Vênus é visto por ...,0,O achado foi publicado nesta quarta-feira (21)...,O achado foi publicado nesta quarta-feira (21)...,A equipe de pesquisa descobriu que o buraco ne...
2,colégio militar,O prefeito estava acompanhado dos secretários ...,“O Colégio Militar tem uma estrutura diferenci...,"Os estudantes, que ainda estão assistindo às a...",0,O prefeito estava acompanhado dos secretários ...,O prefeito estava acompanhado dos secretários ...,O prefeito destacou que a saúde e a segurança ...
3,buraco negro,"O filme tem o protagonismo de Antônio Pitanga,...",Isto o guiará num buraco negro profundo e comp...,A inclusão do filme brasileiro na seleção ofic...,0,"O filme tem o protagonismo de Antônio Pitanga,...","O filme tem o protagonismo de Antônio Pitanga,...","O filme segue essa jornada de Antônio Pitanga,..."
4,bode expiatório,Exclusivo Treinador dos sub-23 do Leixões como...,Vale a pena transformar José Augusto Faria no ...,Vale.,0,Exclusivo Treinador dos sub-23 do Leixões como...,Exclusivo Treinador dos sub-23 do Leixões como...,A resposta é não. É necessário olhar para a ra...


In [38]:
# let's remove the newlines and extra spaces

import re

def remove_newlines_and_extra_spaces(string):
    string = string.replace('\n', ' ')
    string = ' '.join(string.split())               # remove side spaces
    string = re.sub('<[^<]+?>', '', string)         # remove html tags
    string = re.sub(r'([.,!?])\1+', r'\1', string)  # remove repeated punctuation
    return string

df_test['Generated Next'] = df_test['Generated Next'].apply(remove_newlines_and_extra_spaces)
df_test.head()

,MWE,Previous,Target,Next,Label,Context,All,Generated Next
0,sangue quente,E é claro aproveitou a parada para conversar e...,"Após o jogo, o português de uma pequena amostr...",Entrou em campo para reclamar e cobrar o árbit...,0,E é claro aproveitou a parada para conversar e...,E é claro aproveitou a parada para conversar e...,"Ele deu algumas dicas para o jovem atacante, i..."
1,buraco negro,O achado foi publicado nesta quarta-feira (21)...,Os autores da pesquisa apelidaram o buraco neg...,Anel de poeira na órbita de Vênus é visto por ...,0,O achado foi publicado nesta quarta-feira (21)...,O achado foi publicado nesta quarta-feira (21)...,A equipe de pesquisa descobriu que o buraco ne...
2,colégio militar,O prefeito estava acompanhado dos secretários ...,“O Colégio Militar tem uma estrutura diferenci...,"Os estudantes, que ainda estão assistindo às a...",0,O prefeito estava acompanhado dos secretários ...,O prefeito estava acompanhado dos secretários ...,O prefeito destacou que a saúde e a segurança ...
3,buraco negro,"O filme tem o protagonismo de Antônio Pitanga,...",Isto o guiará num buraco negro profundo e comp...,A inclusão do filme brasileiro na seleção ofic...,0,"O filme tem o protagonismo de Antônio Pitanga,...","O filme tem o protagonismo de Antônio Pitanga,...","O filme segue essa jornada de Antônio Pitanga,..."
4,bode expiatório,Exclusivo Treinador dos sub-23 do Leixões como...,Vale a pena transformar José Augusto Faria no ...,Vale.,0,Exclusivo Treinador dos sub-23 do Leixões como...,Exclusivo Treinador dos sub-23 do Leixões como...,A resposta é não. É necessário olhar para a ra...


##BLEU score

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones

def calculate_bleu(label):
    scores=[]

    for i in range(len(df_test)):
        try:
            reference = df_test.loc[df_test['Label'] == label, 'Next'].iloc[i]
            candidate = df_test.loc[df_test['Label'] == label, 'Generated Next'].iloc[i]
            scores.append(sentence_bleu(reference, candidate))
        except IndexError:
            pass

    return statistics.mean(scores)

In [ ]:
# idiomatic [0]
print(calculate_bleu(0))

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1.2296021279608879e-231


In [ ]:
# non-idiomatic [1]
print(calculate_bleu(1))

1.1960903867514186e-231


## Rouge Score

In [ ]:
# Rouge Score
import statistics
from rouge_score import rouge_scorer

def calculate_rouge(label):
    r_scores=[]
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    
    for i in range(len(df_test)):
        try:
            reference = df_test.loc[df_test['Label'] == label, 'Next'].iloc[i]
            candidate = df_test.loc[df_test['Label'] == label, 'Generated Next'].iloc[i]
            scores = scorer.score(reference, candidate)
            r_scores.append(scores)
        except IndexError:
            pass
    
    
    rl_p_scores = []
    rl_r_scores = []
    rl_f_scores = []

    for each_dict in r_scores:
        rl_p_scores.append(each_dict['rougeL'][0])
        rl_r_scores.append(each_dict['rougeL'][1])
        rl_f_scores.append(each_dict['rougeL'][2])


    mean_rl_p = statistics.mean(rl_p_scores)
    mean_rl_r = statistics.mean(rl_r_scores)
    mean_rl_f = statistics.mean(rl_f_scores)

    print('Precision', mean_rl_p)
    print('Recall', mean_rl_r)
    print('F1', mean_rl_f)

In [ ]:
# idiomatic [0]
calculate_rouge(0)

Precision 0.1395297094293344
Recall 0.10307567221181263
F1 0.11067874118690813


In [ ]:
# non-idiomatic [1]
calculate_rouge(1)

Precision 0.1366013944182341
Recall 0.11911454552047956
F1 0.11823399425159266


## CHRF++

In [ ]:
!pip install -q sacrebleu

In [ ]:
def calculate_chrf(label):
    
    # making format compatible
    l = list(df_test.loc[df_test['Label'] == label, 'Next'])
    list_of_list = [[i] for i in l]
    
    chrf = evaluate.load("chrf")
    results = chrf.compute(predictions = list(df_test.loc[df_test['Label'] == label, 'Generated Next']), 
                           references = list_of_list,
                           word_order=2)
    
    # mean_s = statistics.mean(results['score'])

    # print('Score', mean_s)
    print(results['score'])

In [ ]:
calculate_chrf(0)

14.760412345117418


In [ ]:
calculate_chrf(1)

16.66525188076945


## BERT Score

In [8]:
!pip install -q bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.6 MB/s eta 0:00:00


In [16]:
from evaluate import load

def calculate_bs(label):

    
    bertscore = load("bertscore")
    results = bertscore.compute(predictions = list(df_test.loc[df_test['Label'] == label, 'Generated Next']), 
                                references = list(df_test.loc[df_test['Label'] == label, 'Next']), 
                                # model_type = "distilbert-base-uncased")
                                model_type = "microsoft/deberta-xlarge-mnli",
                                lang="pt")
                                # model_type = "roberta-large")
    
    mean_p = statistics.mean(results['precision'])
    mean_r = statistics.mean(results['recall'])
    mean_f = statistics.mean(results['f1'])
    

    print('Precision', mean_p)
    print('Recall', mean_r)
    print('F1', mean_f)

In [39]:
# Idiomatic
calculate_bs(0)

Precision 0.6054084864484162
Recall 0.5962865790900063
F1 0.5992188583902952


In [40]:
# non-idiomatic
calculate_bs(1)

Precision 0.6206876370085388
Recall 0.6124873840007461
F1 0.6157463598652047


## METEOR Score

In [ ]:
# meteor = evaluate.load('meteor')
# predictions = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
# reference = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
# results = meteor.compute(predictions=predictions, references=reference)
# print(round(results['meteor'], 2))

In [ ]:
def calculate_meteor(label):
    scores=[]
    meteor = evaluate.load('meteor')

    for i in range(len(df_test)):
        try:
            reference = list(df_test.loc[df_test['Label'] == label, 'Next'])
            candidate = list(df_test.loc[df_test['Label'] == label, 'Generated Next'])
            
            results = meteor.compute(predictions = candidate, 
                                     references = reference)

            scores.append(results['meteor'])
        except IndexError:
            pass


    mean_m = statistics.mean(scores)
    print('Meteor Score', round(mean_m, 3))

In [ ]:
calculate_meteor(0)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Meteor Score 0.103


In [ ]:
calculate_meteor(1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Meteor Score 0.117
